In [7]:
from ultralytics import YOLO

import os
import shutil
import cv2
# Load the trained YOLOv8 model
model = YOLO("D:/VIT/Garbage Dataset/best.pt")  # Forward slashes


In [13]:
# Define the folder containing images
input_folder = 'D:/VIT/Garbage Dataset/content/test/images'
output_folder = 'D:/VIT/Garbage Dataset/content/test/output'
output_folder_irr='D:/VIT/Garbage Dataset/content/test/output/nonrelevant'
destination_folder='D:/VIT/Garbage Dataset/content/test/images/archive'
target_categories = ['Plastic', 'Cigarette Butt', 'Metal Can', 'Paper', 'Cup']

# Iterate through each image in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_folder, filename)
        archive_image_path=os.path.join(destination_folder, filename)
        output_image_path=os.path.join(output_folder, filename)
   
        # Run model on the image
        results = model(image_path)
        image = cv2.imread(image_path)  # Load the image
       

        a=0
        for result in results:
            # Access bounding box predictions
            for bbox in result.boxes:
                x1, y1, x2, y2 = bbox.xyxy[0].cpu().numpy()  # Bounding box coordinates
                confidence = 100*(bbox.conf.cpu().numpy()) # Get the confidence score
                class_idx = int(bbox.cls.cpu().numpy())  # Get the class label index (numerical)
                class_name = model.names[class_idx]  # Get the class label name from the model's class names
        
                if (class_name in target_categories) and (confidence>50):
                    a=a+1
                    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2) # draw the bounding box on the image
                    cv2.putText(image, f"{class_name} {confidence}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            if a>0:
                cv2.imwrite(output_image_path, image)
            
        shutil.move(image_path, archive_image_path) # move the processed image from input folder to archive
        print(f"Processed and saved: {output_image_path}")



image 1/1 D:\VIT\Garbage Dataset\content\test\images\360_F_1035399369_vpYfZoqEpUNinI3fppu1nCIzjElM63jG.jpg: 448x640 (no detections), 366.0ms
Speed: 15.6ms preprocess, 366.0ms inference, 3.6ms postprocess per image at shape (1, 3, 448, 640)
Processed and saved: D:/VIT/Garbage Dataset/content/test/output\360_F_1035399369_vpYfZoqEpUNinI3fppu1nCIzjElM63jG.jpg

image 1/1 D:\VIT\Garbage Dataset\content\test\images\a-pile-of-garbage-bags-and-other-debris-on-the-side-of-the-road-environmental-disaster-the-collection-of-waste-for-recycling-2ATR725.jpg: 480x640 1 Plastic, 299.2ms
Speed: 0.0ms preprocess, 299.2ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)
Processed and saved: D:/VIT/Garbage Dataset/content/test/output\a-pile-of-garbage-bags-and-other-debris-on-the-side-of-the-road-environmental-disaster-the-collection-of-waste-for-recycling-2ATR725.jpg


C:\Users\Nandika\AppData\Local\Temp\ipykernel_6260\3858175574.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  class_idx = int(bbox.cls.cpu().numpy())  # Get the class label index (numerical)


In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load the pretrained YOLOv8 model (e.g., 'yolov8n.pt' is the nano model)
model = YOLO('runs/detect/train/weights/best.pt')  # Forward slashes

# Open a connection to the webcam (0 is usually the default camera)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Iterate over frames from the webcam
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If the frame is read correctly, ret will be True
    if not ret:
        print("Error: Failed to capture image from webcam.")
        break

    # Run YOLOv8 model on the current frame
    results = model(frame)

    # Get the image with bounding boxes and labels (in numpy array format)
    result_image = results[0].plot()

    # Display the resulting frame
    cv2.imshow('YOLOv8 Webcam Output', result_image)

    # Break the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

print("Webcam processing complete!")
